In [ ]:
import datasets
from datasets import DatasetDict
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# Load dataset
master_dataset = datasets.load_dataset("coastalcph/lex_glue", "scotus")

def sample_subset(dataset_split, sample_ratio=0.1):
    sample_size = int(len(dataset_split) * sample_ratio)
    return dataset_split.shuffle(seed=42).select(range(sample_size))

dataset = DatasetDict({
    "train": sample_subset(master_dataset["train"]),
    "test": sample_subset(master_dataset["test"]),
    "validation": sample_subset(master_dataset["validation"])
})

# Tokenizer setup
tokenizer_legalbert = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
tokenizer_roberta = AutoTokenizer.from_pretrained("roberta-base")

class TextClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(TextClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return self.fc(outputs.pooler_output)

model_legalbert = TextClassifier("nlpaueb/legal-bert-base-uncased", num_labels=13)
model_roberta = TextClassifier("roberta-base", num_labels=13)
metrics_data = []

def train_model(model, tokenizer, dataset, model_name, epochs=1, batch_size=8, lr=2e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    train_encodings = tokenizer(dataset["train"]["text"], truncation=True, padding=True, return_tensors="pt")
    val_encodings = tokenizer(dataset["validation"]["text"], truncation=True, padding=True, return_tensors="pt")
    
    train_labels = torch.tensor(dataset["train"]["label"])
    val_labels = torch.tensor(dataset["validation"]["label"])
    
    train_dataset = torch.utils.data.TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels)
    val_dataset = torch.utils.data.TensorDataset(val_encodings["input_ids"], val_encodings["attention_mask"], val_labels)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        all_preds, all_labels = [], []
        
        for batch in train_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = [b.to(device) for b in batch]
                outputs = model(input_ids, attention_mask)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        metrics_data.append({"Model": model_name, "Epoch": epoch+1, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1})
        print(f"[{model_name}] Epoch {epoch+1}: Loss = {total_loss:.4f}, Accuracy = {accuracy:.4f}")

train_model(model_legalbert, tokenizer_legalbert, dataset, "legal-bert-base-uncased")
train_model(model_roberta, tokenizer_roberta, dataset, "roberta-base")

def train_tfidf_svm(dataset):
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(dataset["train"]["text"])
    X_val_tfidf = vectorizer.transform(dataset["validation"]["text"])
    svm_model = SVC(kernel="linear", probability=True, random_state=42)
    svm_model.fit(X_train_tfidf, dataset["train"]["label"])
    y_pred = svm_model.predict(X_val_tfidf)
    accuracy = accuracy_score(dataset["validation"]["label"], y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(dataset["validation"]["label"], y_pred, average='weighted')
    metrics_data.append({"Model": "TFIDF+SVM", "Epoch": 1, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1})
    print(f"[TFIDF+SVM] Accuracy = {accuracy:.4f}")
    return svm_model, vectorizer

train_tfidf_svm(dataset)
metrics_df = pd.DataFrame(metrics_data)
print(metrics_df)
plt.figure(figsize=(10, 5))
sns.lineplot(data=metrics_df, x="Epoch", y="Accuracy", hue="Model", marker="o")
plt.title("Accuracy per Epoch")
plt.show()